In [ ]:
!pip install tensorflow
!pip install keras

In [10]:
import pandas as pd
import numpy as np

In [ ]:
from keras.models import Sequential

In [ ]:
#library import
import matplotlib.pyplot as plt
from keras.layers import LSTM, Dropout ,Dense, Activation
import datetime

In [1]:
import quote as qt
df_coin = qt.get_coin_quote()
df_coin.head(3)

kind: 1_BTC, 2_ETH 3_SOL
default: KRW-BTC
interval: 1_minute1, 2_minute3, 3_minute5, 4_minute10, 5_minute15, 6_minute30, 7_minute60, 8_minute240, 9_day, 10_week, 11_month
default: minute1
count: #
default: 100


,open,high,low,close,volume,value
2022-04-05 20:46:00,57156000.0,57170000.0,57140000.0,57168000.0,2.217127,1.267379e+08
2022-04-05 20:47:00,57168000.0,57174000.0,57156000.0,57166000.0,1.290954,7.379778e+07
2022-04-05 20:48:00,57158000.0,57173000.0,57156000.0,57158000.0,2.594264,1.482913e+08


# Mid Price

In [4]:
highs = df_coin['high']
lows = df_coin['low']
mids = (highs+lows)/2

# Create Window

In [8]:
period=50
seq_len=period+1

result = []
for index in range(len(mids) - seq_len):
    result.append(mids[index: index + seq_len])

# Nomalize

In [11]:
def normalize_windows(data):
    normalized_data = []
    for window in data:
        normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalized_data.append(normalized_window)
    return np.array(normalized_data)

result = normalize_windows(result)

# split

In [12]:
row = int(round(result.shape[0] * 0.9))
train = result[:row, :]
# 학습효율 -> random
np.random.shuffle(train)

#학습
x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1]

#예측
x_test = result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result[row:, -1]

x_train.shape, x_test.shape

((44, 50, 1), (5, 50, 1))

# Model

In [ ]:
from keras.models import Sequential
model = Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(50, 1)))

model.add(LSTM(64, return_sequences=False))

model.add(Dense(1, activation='linear'))

model.compile(loss='mse', optimizer='rmsprop')

model.summary()